In [4]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data_train = "../fashionmnist/fashion-mnist_train.csv"

# Labels 
labels_map = {  0: "T-shirt/top",
                1: "Trouser",
                2: "Pullover",
                3: "Dress",
                4: "Coat",
                5: "Sandal",
                6: "Shirt",
                7: "Sneaker",
                8: "Bag",
                9: "Ankle boot" }

In [6]:
def get_data(dataframe):
    data = []
    label = []
    for idx, row in dataframe.iterrows():
        data.append(row.values[1:].reshape(-1))
        label.append(row.values[:1].reshape(-1))
    return data, label

In [3]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Using TensorFlow backend.


In [8]:
data_train = "../fashionmnist/fashion-mnist_train.csv"
df = pd.read_csv(data_train)
# Prepare data for the network
x, y = get_data(df)
#Splitting the  data
x_train, x_test, y_train, y_test = train_test_split(np.array(x), np.array(y), test_size=0.33, random_state=42)

In [10]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

print (x_train.shape)
print( x_test.shape)

(40200, 28, 28, 1)
(19800, 28, 28, 1)


In [14]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 40200 samples, validate on 19800 samples
Epoch 1/50
40200/40200 [==============================] - 55s 1ms/step - loss: 0.0363 - val_loss: 0.0090
Epoch 2/50
40200/40200 [==============================] - 52s 1ms/step - loss: 0.0088 - val_loss: 0.0087
Epoch 3/50
40200/40200 [==============================] - 54s 1ms/step - loss: 0.0086 - val_loss: 0.0086
Epoch 4/50
40200/40200 [==============================] - 53s 1ms/step - loss: 0.0085 - val_loss: 0.0085
Epoch 5/50
40200/40200 [==============================] - 56s 1ms/step - loss: 0.0085 - val_loss: 0.0086
Epoch 6/50
40200/40200 [==============================] - 50s 1ms/step - loss: 0.0085 - val_loss: 0.0085
Epoch 7/50
40200/40200 [==============================] - 53s 1ms/step - loss: 0.0085 - val_loss: 0.0085
Epoch 8/50
40200/40200 [==============================] - 52s 1ms/step - loss: 0.0085 - val_loss: 0.0085
Epoch 9/50
40200/40200 [==============================] - 52s 1ms/step - loss: 0.0085 - val_loss: 0.0085
Epoch

In [18]:
# x_train

In [15]:
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

ValueError: num must be 1 <= num <= 20, not 0

<Figure size 1440x288 with 0 Axes>

In [16]:
decoded_imgs

array([[[[6.19680068e-05],
         [3.37397978e-05],
         [5.33306848e-05],
         ...,
         [7.45270881e-05],
         [7.55403235e-05],
         [1.41542609e-04]],

        [[1.80421139e-05],
         [1.62964880e-05],
         [3.30351795e-05],
         ...,
         [9.63014754e-05],
         [8.70904332e-05],
         [3.17523663e-05]],

        [[2.50786725e-05],
         [2.35314255e-05],
         [4.88692931e-05],
         ...,
         [1.11161964e-04],
         [8.22922520e-05],
         [2.98163613e-05]],

        ...,

        [[1.38761112e-04],
         [1.90108738e-04],
         [3.18611623e-04],
         ...,
         [4.72710875e-04],
         [2.81096378e-04],
         [1.19967626e-04]],

        [[1.34001020e-04],
         [1.69231222e-04],
         [2.60089466e-04],
         ...,
         [3.68174340e-04],
         [2.38336419e-04],
         [1.05492720e-04]],

        [[1.34489761e-04],
         [9.29689850e-05],
         [1.32468995e-04],
         ...,
 